## Importing Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import re
import nltk
nltk.download('wordnet_ic')
from nltk import pos_tag, word_tokenize, PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

wordnet_lemmatizer = WordNetLemmatizer()
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

## Reading the Csv file and Dropping Null Values

In [ ]:
df=pd.read_csv("../input/billboard_lyrics_1964-2015.csv",encoding = 'cp1252')
df.dropna(axis=0,inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

#### Changing the type of Year to string

In [ ]:
df['Year']=df['Year'].astype('str')

# USING KNN FROM SCRATCH TO RECOMMEND SONGS ON THE BASIS OF ARTIST'S NAME AND SONG LYRICS.

In [ ]:
use_cols=df['Lyrics']
use_cols.head()

In [ ]:
use_cols.head(50)
lyrics_list=[]
for i in range(4913):
    try:
        lyrics_list.append(use_cols[i])
    except KeyError:
        continue
lyrics_list

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer 
tfidf_vectorizer=TfidfVectorizer(use_idf=True,stop_words= 'english')
tfidf_vectorizer_vectors=tfidf_vectorizer.fit_transform(lyrics_list)
tfidf_vectorizer_vectors.shape

In [ ]:
# get the first vector out (for the first document)
first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[0]
 # place tf-idf values in a pandas data frame
df_lyrics = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
df_lyrics =df_lyrics.sort_values(by=["tfidf"],ascending=False)
df_lyrics 

In [ ]:
print(tfidf_vectorizer_vectors.shape)

In [ ]:
use_cols2=df['Artist']
use_cols2.head()

In [ ]:
artist_list=[]
for i in range(4913):
    try:
        artist_list.append(use_cols2[i])
    except KeyError:
        continue
artist_list

In [ ]:
tfidf_vectorizer2=TfidfVectorizer(use_idf=True,stop_words= 'english')
tfidf_vectorizer_vectors2=tfidf_vectorizer2.fit_transform(artist_list)
print(tfidf_vectorizer_vectors2.shape)

In [ ]:
# get the first vector out (for the first document)
first_vector_tfidfvectorizer2=tfidf_vectorizer_vectors2[0]
# place tf-idf values in a pandas data frame
df_artist = pd.DataFrame(first_vector_tfidfvectorizer2.T.todense(), index=tfidf_vectorizer2.get_feature_names(), columns=["tfidf2"])
df_artist =df_artist.sort_values(by=["tfidf2"],ascending=False)
df_artist


In [ ]:
final=[]
f2=[]
for i in range(4729):
    first_vector_tfidfvectorizer=tfidf_vectorizer_vectors[i]
    df_lyrics = pd.DataFrame(first_vector_tfidfvectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=["tfidf"])
    df_lyrics =df_lyrics.sort_values(by=["tfidf"],ascending=False)
    first_vector_tfidfvectorizer2=tfidf_vectorizer_vectors2[i]
    df_artist = pd.DataFrame(first_vector_tfidfvectorizer2.T.todense(), index=tfidf_vectorizer2.get_feature_names(), columns=["tfidf2"])
    df_artist =df_artist.sort_values(by=["tfidf2"],ascending=False)
    for j in range(4):
        f2.append(df_artist.iloc[j,0])
    for k in range(6):
        f2.append(df_lyrics.iloc[k,0])
    final.append(f2)
    f2=[]
final_df=pd.DataFrame(final)
final_df.head()    

In [ ]:
use_cols3=df['Song'].iloc[0:4729,]
print(use_cols3.head())

In [ ]:
y=pd.DataFrame(use_cols3.index)
y.head()

In [ ]:
x=final_df
x.head()

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.8,random_state=42)

## KNN from Scratch

In [ ]:
def knnpredict(x_train,x_test,y_train,k):
    final=[]
    diff=[]
    ma=[]
    for i in range(len(x_train)):
        dist=np.sqrt(np.sum(np.square(x_train.values[i]-x_test)))
        diff.append((dist,y_train.values[i]))
        
        
    diff=sorted(diff,key=lambda x: x[0])
    for i in range(k):
        ma.append(diff[i][1])
    
    return ma

In [ ]:
a=[]
for i in range(len(y_test)):
    
    a.append(knnpredict(x_train,x_test.values[i,:],y_train,10))
    print("10 recommended songs for ")
    print(use_cols3[y_test.iloc[i,:]])
    print("is-")
    for j in range(10):
        print(use_cols3[a[i][j]])
        
    print("------------------------")   

# Using inbuilt KNN Methods to Recommend Songs on basis of Lyrics .

In [ ]:
tfid= TfidfVectorizer(stop_words='english')
data= tfid.fit_transform(df['Lyrics'])
print(data.shape)


In [ ]:
doc=list((df['Lyrics']).values)
data_l= tfid.fit_transform(doc)
nn= NearestNeighbors(algorithm= 'brute', n_neighbors= 12).fit(data_l)

In [ ]:
text= df[df['Song']=='mrs brown youve got a lovely daughter']['Lyrics'].values
result = nn.kneighbors(tfid.transform(text))
for col in tfid.transform(text).nonzero()[1]:
    print(tfid.get_feature_names()[col], ' - ', tfid.transform(text)[0, col])


In [ ]:
for item in result[1][0]:
    print((df.iloc[item]['Song'].upper()))